In [1]:
#based on https://stackoverflow.com/questions/43101497/how-do-stream-data-to-a-bokeh-plot-in-jupyter-with-a-high-refresh-rate

import time
import os
import signal
from threading import Thread, Event

from sh import tail

import numpy as np

from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import ColumnDataSource, Range1d, LinearAxis
from bokeh.io import output_notebook, reset_output, show, push_notebook

from ktdashboard.ktdashboard import KTdashboard

# reset_output()
output_notebook()


Loading BokehJS ...

In [5]:
height = 500
width = 800
WITH_KT = True

fig_power = figure(
        height=height,
        width=width,
        title="PowerSensor3 measurements",
        y_range=(0, 12),
        toolbar_location=None
    )
fig_power.xaxis.axis_label = "Time (s)"
fig_power.yaxis.axis_label = "Power (W)"

fig_gpu = figure(
        height=height,
        width=width,
        title="GPU status",
        y_range=(0, 1000),
        toolbar_location=None
    )
fig_gpu.xaxis.axis_label = "Time (s)"
fig_gpu.yaxis.axis_label = "GPU Frequency (MHz)"
fig_gpu.extra_y_ranges = {"load": Range1d(start=-5, end=105)}
fig_gpu.add_layout(LinearAxis(y_range_name="load", axis_label="GPU load (%)"), 'right')

data = ColumnDataSource(data={'time': [], 'power_ps': [], 'power_ps_min': [], 'power_ps_max': [], 'power_tegra_cpu': [], 'power_tegra_gpu': [], 'gpu_freq': [], 'gpu_load': []})

fig_power.line("time", "power_ps", source=data, color="blue", legend_label="PowerSensor3", line_width=2)
fig_power.line("time", "power_ps_min", source=data, alpha=.5, color="blue", line_dash="dashed")
fig_power.line("time", "power_ps_max", source=data, alpha=.5, color="blue", line_dash="dashed")

fig_power.line("time", "power_tegra_cpu", source=data, color='red', legend_label="Tegra - CPU", line_width=2)
fig_power.line("time", "power_tegra_gpu", source=data, color='green', legend_label="Tegra - GPU", line_width=2)

fig_gpu.line("time", "gpu_freq", source=data, legend_label="Frequency", line_width=2)
fig_gpu.line("time", "gpu_load", source=data, color='red', legend_label="Load", line_width=1, alpha=.5, y_range_name="load")

fig_power.legend.location = "top_left"
fig_gpu.legend.location = "top_left"

if WITH_KT:
    # add the kernel tuner dashboard, with a custom figure
    ktdash = KTdashboard("/home/powersensor/PowerSensor/powersensor_demo/dedisp_cache.json", demo=False)
    fig_kt = figure(
            height=height,
            width=width,
            title="Kernel Tuner",
            toolbar_location=None
    )
    
    kt_xvariable = "GB/s/W (system)"
    kt_yvariable = "GB/s"
    kt_colorvariable = "GPU frequency (MHz)"
    fig_kt.xaxis.axis_label = kt_xvariable
    fig_kt.yaxis.axis_label = kt_yvariable
    plt_kt = fig_kt.scatter(kt_xvariable, kt_yvariable, size=12, color=ktdash.update_colors(kt_colorvariable), alpha=.5, source=ktdash.source)
    cbar = plt_kt.construct_color_bar(padding=1)
    # separate fig for colorbar so we can add a label
    fig_cbar = figure(height=height, width=100, title="GPU frequency (MHz)", title_location="right", toolbar_location=None,
                     min_border=0, outline_line_color=None)
    fig_cbar.add_layout(cbar, "right")
    fig_cbar.title.align = "center"

if WITH_KT:
    handle = show(row(fig_power, fig_gpu, fig_kt, fig_cbar), notebook_handle=True)
else:
    handle = show(row(fig_power, fig_gpu), notebook_handle=True)

In [3]:
event = Event()

def update_sensors(_, event):
    n_sec_to_show = 30
    # powersensor dumpefile freq reduced by a factor 2000: so 10 Hz
    ps_freq = 10
    n_show = n_sec_to_show * ps_freq

    # wait for file to exist
    fname = '/dev/shm/ps3.txt'
    timeout = 9999  # seconds
    tstart = time.time()
    while not os.path.exists(fname):
        event.wait(.1)
        if time.time() - tstart > timeout:
            print(f"File {fname} still does not exist, giving up")
            return

    print("Starting read")

    process = tail('-F', '-n 0', '/dev/shm/ps3.txt', _iter=True)
    # first_offset = None
    for line in process:
        if not line.startswith('S'):
            continue

        line = line.split(' ')
        t = float(line[1])
        p, power_tegra_cpu, power_tegra_gpu, gpu_freq, gpu_load, p_min, p_max = (float(item) for item in line[-7:])
        
        data.stream({'time': [t], 'power_ps': [p], 'power_tegra_gpu': [power_tegra_gpu], 'power_tegra_cpu': [power_tegra_cpu],
                     'gpu_freq': [gpu_freq], 'gpu_load': [gpu_load],
                     'power_ps_min': [p_min], 'power_ps_max': [p_max]},
                    n_show)
        
        if event.is_set():
            # print(offset)
            process.signal(signal.SIGINT)
            return
            
    print("Sensor update stopped")

def update_figs(id, event):
    while not event.is_set():
        push_notebook(handle=handle)
        event.wait(.1)
    print("Figure update stopped")

def update_kt(id, event):
    while not event.is_set():
        ktdash.update_data()
        event.wait(5)
    print("Kernel tuner update stopped")
        


if WITH_KT:
    funcs = (update_sensors, update_kt, update_figs)
else:
    funcs = (update_sensors, update_figs)

threads = []
for func in funcs:
    threads.append(Thread(target=func, args=(id, event)))

for t in threads:
    t.start()

Starting read


In [4]:
# event.set()
# for t in threads:
#     t.join()